In [7]:
import numpy as np
import xgboost as xgb

data = np.loadtxt('data/dermatology.data',delimiter=',',converters={33:lambda x:int(x=='?'),34:lambda x:int(x)-1})
data

array([[ 2.,  2.,  0., ...,  0.,  0.,  1.],
       [ 3.,  3.,  3., ...,  0.,  0.,  0.],
       [ 2.,  1.,  2., ...,  3.,  0.,  2.],
       ..., 
       [ 3.,  2.,  2., ...,  3.,  0.,  2.],
       [ 2.,  1.,  3., ...,  3.,  0.,  2.],
       [ 3.,  2.,  2., ...,  0.,  0.,  0.]])

In [12]:
sz = data.shape
train = data[:int(sz[0]*0.7),:]  # 选择 70% 作为训练数据
test = data[int(sz[0]*0.7):,:]  # 剩下的 30% 作为测试数据集

train_X = train[:,0:33]
train_Y = train[:,34]

test_X = test[:,0:33]
test_Y = test[:,34]

In [13]:
#  构建训练模型数据
xg_train = xgb.DMatrix(train_X,label=train_Y)
xg_test = xgb.DMatrix(test_X,label=test_Y)

In [14]:
# 构建训练参数
param = {}
# 使用 softmax multi-class 作为分类器
param['objective']='multi:softmax'

param['eta'] = 0.1  # 步长
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6

watchlist = [(xg_train,'train',(xg_test,'test'))]
num_round = 5
bst = xgb.train(param,xg_train,num_round,watchlist)

# 获得模型
pred = bst.predict(xg_test)

print ('predicting, classification error=%f' % (sum( int(pred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

[0]	train-merror:0.011719
[1]	train-merror:0.015625
[2]	train-merror:0.011719
[3]	train-merror:0.007812
[4]	train-merror:0.007812
predicting, classification error=0.090909


In [15]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist );
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], 6 )
ylabel = np.argmax(yprob, axis=1)

print ('predicting, classification error=%f' % (sum( int(ylabel[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

[0]	train-merror:0.011719
[1]	train-merror:0.015625
[2]	train-merror:0.011719
[3]	train-merror:0.007812
[4]	train-merror:0.007812
predicting, classification error=0.090909
